## 导入相关工具和库

In [1]:
import os
import random
from pathlib import Path

from PIL import Image

## 处理数据

In [2]:
SRC_DIR = Path("FullIJCNN2013")         
GT_PATH = SRC_DIR / "gt.txt"

OUT_DIR = Path("gtsdb_yolo")            # 输出YOLO数据集
TRAIN_RATIO = 0.8
SEED = 42
IMG_EXT = ".png"                        # 转成png

In [3]:
def ensure_dirs():
    for p in [
        OUT_DIR / "images/train", OUT_DIR / "images/val",
        OUT_DIR / "labels/train", OUT_DIR / "labels/val",
    ]:
        p.mkdir(parents=True, exist_ok=True)

def read_gt(gt_path: Path):
    # gt.txt 每行: filename;x1;y1;x2;y2;classId
    ann = {}  # filename -> list of (x1,y1,x2,y2)
    with gt_path.open("r", encoding="utf-8", errors="ignore") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            parts = line.split(";")
            if len(parts) < 6:
                continue
            fname, x1, y1, x2, y2, _cls = parts[:6]
            x1, y1, x2, y2 = map(int, [x1, y1, x2, y2])
            ann.setdefault(fname, []).append((x1, y1, x2, y2))
    return ann

def convert_one_image(ppm_path: Path, out_img_path: Path):
    # ppm -> png
    img = Image.open(ppm_path)
    img.save(out_img_path)
    return img.size  # (W, H)

def write_yolo_label(boxes, w, h, label_path: Path):
    # 所有类别统一为 0: traffic sign
    lines = []
    for (x1, y1, x2, y2) in boxes:
        # 确保坐标合法
        x1 = max(0, min(x1, w - 1))
        x2 = max(0, min(x2, w - 1))
        y1 = max(0, min(y1, h - 1))
        y2 = max(0, min(y2, h - 1))
        if x2 <= x1 or y2 <= y1:
            continue

        xc = ((x1 + x2) / 2.0) / w
        yc = ((y1 + y2) / 2.0) / h
        bw = (x2 - x1) / w
        bh = (y2 - y1) / h
        lines.append(f"0 {xc:.6f} {yc:.6f} {bw:.6f} {bh:.6f}")

    label_path.write_text("\n".join(lines) + ("\n" if lines else ""), encoding="utf-8")

In [4]:
def main():
    ensure_dirs()
    ann = read_gt(GT_PATH)

    # 全部图片名（以 gt.txt 出现过的为准）
    all_imgs = sorted(ann.keys())
    random.seed(SEED)
    random.shuffle(all_imgs)

    n_train = int(len(all_imgs) * TRAIN_RATIO)
    train_set = set(all_imgs[:n_train])
    val_set = set(all_imgs[n_train:])

    for fname in all_imgs:
        split = "train" if fname in train_set else "val"

        ppm_path = SRC_DIR / fname
        if not ppm_path.exists():
            # 有的gt行可能不完整/或文件名异常，直接跳过
            continue

        out_img_path = OUT_DIR / f"images/{split}/{Path(fname).stem}{IMG_EXT}"
        out_label_path = OUT_DIR / f"labels/{split}/{Path(fname).stem}.txt"

        w, h = convert_one_image(ppm_path, out_img_path)
        write_yolo_label(ann[fname], w, h, out_label_path)

    # 写 dataset yaml
    yaml_text = f"""path: {OUT_DIR.resolve()}
train: images/train
val: images/val

nc: 1
names: ["traffic sign"]
"""
    (OUT_DIR / "gtsdb.yaml").write_text(yaml_text, encoding="utf-8")
    print("Done! Dataset prepared at:", OUT_DIR)

if __name__ == "__main__":
    main()

Done! Dataset prepared at: gtsdb_yolo
